In [16]:
import azureml.core
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Experiment, Workspace
import azureml.core
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Experiment, Workspace

# Check core SDK version number
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")
print("")

# Log In to Azure ML Workspace
interactive_auth = InteractiveLoginAuthentication(tenant_id="76f90eb1-fb9a-4446-9875-4d323d6455ad")

ws = Workspace.from_config(auth=interactive_auth)
print('Workspace name: ' + ws.name, sep='\n')
from azureml.core import Dataset

# get the dataset from Azure ML Workspace
aml_dataset = Dataset.get_by_name(ws, 'test_ds')
test_dataset = aml_dataset.to_pandas_dataframe()

aml_dataset = Dataset.get_by_name(ws, 'dataset_final')
train_dataset = aml_dataset.to_pandas_dataframe()

You are currently using version 1.5.0 of the Azure ML SDK

Workspace name: team15


In [17]:
test_dataset.to_csv('test.csv')

In [18]:
import lightgbm
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm_notebook
import gc
from sklearn.model_selection import train_test_split

In [19]:
def custom_metric(answers, take_top_ratio=0.25):
    answers.sort_values(by='uplift', inplace=True, ascending=False)
    n_samples = int(np.ceil(answers.shape[0] * take_top_ratio))
    answers = answers.iloc[:n_samples, :]
    answers_test = answers[answers['group'] == 'test']['response_att'].sum() / \
                   answers[answers['group'] == 'test'].shape[0]
    answers_control = answers[answers['group'] == 'control']['response_att'].sum() / \
                      answers[answers['group'] == 'control'].shape[0]
    return (answers_test - answers_control) * 100


In [20]:
train_dataset = train_dataset.drop('Column1', axis=1)

In [21]:
y = train_dataset.group == train_dataset.response_att
train = train_dataset.drop(['group', 'response_att'], axis=1)
train = train.set_index('CardHolder')

In [22]:
y.sum()

210395

In [23]:
for feature in train_dataset:
    if '_div_' in feature:
        col_1, col_2 = feature.split('_div_')
        test_dataset[feature] = test_dataset[col_1]/test_dataset[col_2]
    if '_mul_' in feature:
        col_1, col_2 = feature.split('_mul_')
        test_dataset[feature] = test_dataset[col_1]*test_dataset[col_2]

In [24]:
test_data = test_dataset[train_dataset.drop(['group', 'response_att'], axis=1).columns]

In [25]:
test_data = test_data.set_index('CardHolder')

In [ ]:
uplift_test = np.zeros(test_data.shape[0])
train_score = 0
train_uplift = np.zeros(train.shape[0])
max_score = 0
max_uplift = None
scores = []
uplifts = np.zeros((100, test_data.shape[0]))
for i in tqdm_notebook(range(10)):
    seed = np.random.choice(10000000000)
    model = LGBMClassifier(boosting_type='dart', max_depth=3, 
                       n_estimators=65, class_weight='balanced', n_jobs=-1, random_state=seed, learning_rate=0.5, reg_lambda=0.1,
                          )
    model.fit(train, y)
    train_uplift += model.predict_proba(train)[:, 1]
    ans = pd.DataFrame({
        'uplift': model.predict_proba(train)[:, 1],
        'response_att': train_dataset.response_att,
        'group': train_dataset.group.map({1: 'test', 0: 'control'})
    })
    score = custom_metric(ans, 0.5)
    print(score)
    train_score += score
    uplift_test += model.predict_proba(test_data)[:, 1]
    uplifts[i] = model.predict_proba(test_data)[:, 1]
    scores.append(score)
    if score > max_score:
        score = max_score
        max_uplift = model.predict_proba(test_data)[:, 1]
uplift_test_1 = uplift_test
train_uplift_1 = train_uplift

2.506436509210372
2.4857005392302343
2.500328309032582
2.513254617104932
2.501902557824509
2.509692188533635
2.5383721828880406
2.5068707988853802
2.4877039309408016
2.500542977053613


In [57]:
np.argsort(scores
)

array([1, 3, 2, 0, 6, 7, 5, 4, 9, 8])

In [ ]:
final_uplift = uplifts[np.argsort(scores)[:-1]].sum(axis=0)
final_uplift.shape

In [84]:
uplift_test = np.zeros(test_data.shape[0])
train_score = 0
train_uplift = np.zeros(train.shape[0])
for i in tqdm_notebook(range(40)):
    seed = np.random.choice(10000000000)
    model = LGBMClassifier(boosting_type='dart', max_depth=2, 
                       n_estimators=105, class_weight='balanced', n_jobs=-1, random_state=seed)
    model.fit(train, y)
    train_uplift += model.predict_proba(train)[:, 1]
    ans = pd.DataFrame({
        'uplift': model.predict_proba(train)[:, 1],
        'response_att': train_dataset.response_att,
        'group': train_dataset.group.map({1: 'test', 0: 'control'})
    })
    score = custom_metric(ans)
    print(score)
    train_score += score
    uplift_test += model.predict_proba(test_data)[:, 1]
uplift_test_2 = uplift_test
train_uplift_2 = train_uplift

6.208766247520731
6.182047157819609
6.152271588054134
6.139356142347488
6.189666727433263
6.1984265355588235
6.178920913295744
6.204599772806457
6.168834824896141
6.2153087651122485
6.146280089865014
6.20912372816598
6.167727138319656
6.175259340354897
6.180770723842441
6.164909468996835
6.168404024999
6.187188767682317
6.214573204028396
6.196489697776625
6.199802466368784
6.131650608545184
6.173882722116505
6.179771879678634
6.204014468614016
6.15753379138145
6.178787178585746
6.156355055760415
6.200984156360187
6.174154829908765
6.130534280905497
6.1916795418763275
6.168533870581117
6.181349282647303
6.188720654015157
6.14498302321917
6.158116582767795
6.171945478010016
6.185583691449023
6.16740710708977



In [79]:
train_score/40

6.175773721779237

In [85]:
uplift_test = np.zeros(test_data.shape[0])
train_score = 0
train_uplift = np.zeros(train.shape[0])

for i in tqdm_notebook(range(40)):
    seed = np.random.choice(10000000000)
    model = LGBMClassifier(boosting_type='gbdt', max_depth=3, 
                       n_estimators=35, class_weight='balanced', n_jobs=-1, random_state=seed)
    model.fit(train, y)
    train_uplift += model.predict_proba(train)[:, 1]
    ans = pd.DataFrame({
        'uplift': model.predict_proba(train)[:, 1],
        'response_att': train_dataset.response_att,
        'group': train_dataset.group.map({1: 'test', 0: 'control'})
    })
    score = custom_metric(ans)
    print(score)
    train_score += score
    uplift_test += model.predict_proba(test_data)[:, 1]
uplift_test_3 = uplift_test
train_uplift_3 = train_uplift

6.228245795757867
6.205378626881491
6.216262761801278
6.208164706873964
6.225729972157598
6.26021029380657
6.161516258169225
6.215725958909346
6.194576994597509
6.201995345004563
6.228932056610029
6.189783295436638
6.178909732581072
6.2505199140674765
6.225439775374516
6.213529885820049
6.184601040638263
6.188897586065415
6.170032259398117
6.234052813456548
6.232318769560652
6.205731649623014
6.241827414966253
6.231878573671554
6.241850235865779
6.213643299660424
6.217663715283689
6.217837078873334
6.169745511214756
6.240086315189139
6.199007221466654
6.192229842672051
6.234709080209505
6.179747018797224
6.208281565436816
6.216673953677143
6.195001195215896
6.231516525862846
6.228933773853293
6.236239777123287



In [86]:
train_score/40

6.21293568979077

In [99]:
ans = pd.DataFrame({
    'uplift': train_uplift_1,
    'response_att': train_dataset.response_att,
    'group': train_dataset.group.map({1: 'test', 0: 'control'})
})
custom_metric(ans)

6.266296501576049

In [90]:
ans = pd.DataFrame({
    'uplift': train_uplift_2 ,
    'response_att': train_dataset.response_att,
    'group': train_dataset.group.map({1: 'test', 0: 'control'})
})
custom_metric(ans)

6.202564921995313

In [91]:
ans = pd.DataFrame({
    'uplift': train_uplift_3 ,
    'response_att': train_dataset.response_att,
    'group': train_dataset.group.map({1: 'test', 0: 'control'})
})
custom_metric(ans)

6.22252454514936

In [96]:
ans = pd.DataFrame({
    'uplift': train_uplift_3 + train_uplift_1 ,
    'response_att': train_dataset.response_att,
    'group': train_dataset.group.map({1: 'test', 0: 'control'})
})
custom_metric(ans)

6.2567074257407596

In [ ]:
pd.DataFrame({'CardHolder': test_data.reset_index().CardHolder, 'uplift': final_uplift}).to_csv('submission_14.csv', index=0, 
                                                                                               sep=';')

In [76]:
pd.DataFrame({'CardHolder': test_data.reset_index().CardHolder, 'uplift': uplift_test})

,CardHolder,uplift
0,16400802,22.387170
1,15752880,16.375971
2,15978290,18.474858
3,16604118,17.416374
4,15880709,17.720552
5,16434752,27.885253
6,16327231,26.475865
7,16021120,27.816648
8,16569670,26.221466
9,16395774,28.115676


(297417, 9)